In [1]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path = '.venv/.env_prod')

True

In [2]:
from sumup import Sumup
from sumup.checkouts import CreateCheckoutBody, ProcessCheckoutBody
import uuid

In [3]:
os.getenv("sumup_api_key")

'sup_sk_6lhjzXUWCcXk9zJqabc4xtTRXBnT5y5tn'

In [4]:
import django
from asgiref.sync import sync_to_async

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "site_reservation.settings")
django.setup()
from reservations.utils import *

In [5]:
BASE_URL = "https://api.sumup.com"

In [6]:
os.getenv("site_domain")

'mesle-entreprises.fr'

In [7]:
sumup_api_key = os.getenv("sumup_api_key")
client = Sumup(api_key=sumup_api_key)

In [8]:
merchant = client.merchant.get().merchant_profile.merchant_code
print(f"Merchant: {merchant} vs {os.getenv('merchant_code_official')}")

Merchant: MCL3GUEQ vs MCL3GUEQ


/tmp/ipykernel_16305/881049463.py:1: DeprecationWarning: The _Retrieve a profile_ endpoint is deprecated, please use the `Merchant` object instead (see [Merchants](https://developer.sumup.com/api/merchants)).
  merchant = client.merchant.get().merchant_profile.merchant_code


In [9]:
checkout = create_checkout(sumup_api_key, os.getenv('merchant_code_official'),1.0, "" )


        🟠 Fonction - CREATE CHECKOUT
        Checkout ID: 21818f72-0999-4ff6-8fb6-595a91a8f079
        Checkout Reference: ODDUYF
          


In [11]:
import os

from sumup import Sumup
from sumup.checkouts.resource import CreateCheckoutBody

checkout = client.checkouts.create(
    CreateCheckoutBody(
        merchant_code=os.environ["merchant_code_official"],
        amount=25.00,
        checkout_reference="ORDER-1002",
        currency="EUR",
        description="Online payment via card widget",
    )
)

print(checkout.id)
# Return checkout.id to your webpage so the SumUp card widget can complete the payment.

b94b5b8a-39f6-464e-bfcc-78a490239572


In [ ]:
checkout_id = "d1a0f7d8-e969-4f34-9067-2d7b4c5b8526" #d1a0f7d8-e969-4f34-9067-2d7b4c5b8526

In [38]:
client.checkouts.get(checkout_id)

CheckoutSuccess(amount=2.2, checkout_reference='fceb1a73', currency='EUR', customer_id=None, date=datetime.datetime(2025, 12, 11, 20, 59, 23, 54000, tzinfo=TzInfo(0)), description='description', id='d1a0f7d8-e969-4f34-9067-2d7b4c5b8526', mandate=None, merchant_code='MCL3GUEQ', merchant_name='MESLÉ VTC', payment_instrument=None, redirect_url='http://127.0.0.1:8000/paiement/resultat/', return_url='http://127.0.0.1:8000/webhook/', status='PAID', transaction_code='TAAAYVVXPC7', transaction_id='a69ae715-b9f1-42fb-81d6-37a8f7f2f762', transactions=[CheckoutSuccessTransaction(amount=2.2, auth_code='153806', currency='EUR', entry_mode='CUSTOMER_ENTRY', id='a69ae715-b9f1-42fb-81d6-37a8f7f2f762', installments_count=1, internal_id=9959083090, merchant_code='MCL3GUEQ', payment_type='ECOM', status='SUCCESSFUL', timestamp=datetime.datetime(2025, 12, 11, 21, 3, 13, 42000, tzinfo=TzInfo(0)), tip_amount=None, transaction_code='TAAAYVVXPC7', vat_amount=None)], valid_until=None)

In [ ]:
""" PAIEMENT EFFECTUE ET VALIDE
CheckoutSuccess(
amount=2.2,
checkout_reference='fceb1a73',
currency='EUR',
customer_id=None,
date=datetime.datetime(2025, 12, 11, 20, 59, 23, 54000, tzinfo=TzInfo(0)),
description='description',
id='d1a0f7d8-e969-4f34-9067-2d7b4c5b8526',
mandate=None,
merchant_code='MCL3GUEQ',
merchant_name='MESLÉ VTC',
payment_instrument=None,
redirect_url='http://127.0.0.1:8000/paiement/resultat/',
return_url='http://127.0.0.1:8000/webhook/',
status='PAID',
transaction_code='TAAAYVVXPC7',
transaction_id='a69ae715-b9f1-42fb-81d6-37a8f7f2f762',
transactions=[
    CheckoutSuccessTransaction(
    amount=2.2,
    auth_code='153806', 
    currency='EUR', 
    entry_mode='CUSTOMER_ENTRY', 
    id='a69ae715-b9f1-42fb-81d6-37a8f7f2f762', 
    installments_count=1, 
    internal_id=9959083090, 
    merchant_code='MCL3GUEQ', 
    payment_type='ECOM', 
    status='SUCCESSFUL', 
    timestamp=datetime.datetime(2025, 12, 11, 21, 3, 13, 42000, tzinfo=TzInfo(0)), 
    tip_amount=None, 
    transaction_code='TAAAYVVXPC7', 
    vat_amount=None)
    ],
    valid_until=None)
"""

In [32]:
client.checkouts.list()

[]

In [12]:
client.checkouts.list_available_payment_methods(merchant_code)

GetPaymentMethods200Response(available_payment_methods=[GetPaymentMethods200ResponseAvailablePaymentMethod(id='apple_pay'), GetPaymentMethods200ResponseAvailablePaymentMethod(id='google_pay'), GetPaymentMethods200ResponseAvailablePaymentMethod(id='card')])

In [26]:
client.checkouts

In [ ]:

def get_transaction_id(checkout_id: str, sumup_api_key: str) -> str | None:
    """
    Récupère un checkout SumUp via son id_checkout
    et renvoie le transaction_id associé.
    """
    client = Sumup(api_key=sumup_api_key)
    # Récupération du checkout
    checkout = client.checkout.get(checkout_id)

    # Même logique que ci-dessus
    if checkout.transaction_id:
        return checkout.transaction_id

    if checkout.transactions:
        return checkout.transactions[0].id

    return None


In [ ]:
def refund_partial(transaction_id: str, original_amount: float, ratio: float = 0.8):
    """Remboursement partiel (80% par défaut)."""
    refund_amount = round(original_amount * ratio, 2)

    url = f"{BASE_URL}/v0.1/me/refund/{transaction_id}"
    headers = {
        "Authorization": f"Bearer {sumup_api_key}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json={"amount": refund_amount})

    print("\n=== REMBOURSEMENT PARTIEL ===")
    print(f"Montant remboursé : {refund_amount} EUR")
    print(f"HTTP {response.status_code}")

    if response.status_code == 204:
        print("✔ Remboursement 80% OK")
    else:
        print(response.text)

    return response


def refund_full(transaction_id: str):
    """Remboursement total."""
    url = f"{BASE_URL}/v0.1/me/refund/{transaction_id}"
    headers = {"Authorization": f"Bearer {sumup_api_key}"}

    response = requests.post(url, headers=headers)

    print("\n=== REMBOURSEMENT TOTAL ===")
    print(f"HTTP {response.status_code}")

    if response.status_code == 204:
        print("✔ Remboursement total OK")
    else:
        print(response.text)

    return response

In [ ]:
# 3) Remboursement à 80%
refund_partial(transaction_id, price)

In [37]:
# 4) Remboursement total
refund_full("a69ae715-b9f1-42fb-81d6-37a8f7f2f762")


=== REMBOURSEMENT TOTAL ===
HTTP 204
✔ Remboursement total OK


<Response [204]>